In [1]:
import pandas as pd
from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import numpy as np
from functools import reduce
# Reading csv data
dataEl_2005_06 = pd.read_csv('data/school/elementary_2005_06.csv', encoding='utf-8')
dataEl_2006_07 = pd.read_csv('data/school/elementary_2006_07.csv', encoding='utf-8')
dataEl_2007_08 = pd.read_csv('data/school/elementary_2007_08.csv', encoding='utf-8')
dataEl_2008_09 = pd.read_csv('data/school/elementary_2008_09.csv', encoding='utf-8')
dataEl_2009_10 = pd.read_csv('data/school/elementary_2009_10.csv', encoding='utf-8')
dataEl_2010_11 = pd.read_csv('data/school/elementary_2010_11.csv', encoding='utf-8')
dataEl_2011_12 = pd.read_csv('data/school/elementary_2011_12.csv', encoding='utf-8')



In [2]:
geography = pd.read_csv('data/geography.csv',encoding='UTF-8')
geo = geography[['geo_level','geo_code','name']]

# Calculating country data
def setCountryData(df,cols):
    for x in cols:
         df.loc[df['geo_level'] == 'country', x]= df.loc[df['geo_level'] == 'state', x].sum()
    return df;

def getGeoData(df):
    cols = list(df.columns.values)[3:]
    df = df.replace(',','',regex=True)
    df[cols] = df[cols].astype(float)
    df = df.rename(index=str, columns={'STATE NAME': 'name'})
    dfState = df.groupby(['name'])[cols].sum().reset_index()
   
    dfDis   = df.groupby(['DISTRICT NAME'])[cols].sum().reset_index()
    dfDis   = dfDis.rename(index=str, columns={'DISTRICT NAME': 'name'})
    dfDis['name'] = dfDis['name'].str.title()
    
    dfCom = pd.concat([dfState, dfDis])
    dfgeo = geo.merge(dfCom ,on=['name'],how='left')
    dfgeo = dfgeo.sort_values(by=['geo_level','name'])
    dfgeo = dfgeo.fillna(method = 'bfill', axis=0).fillna(0)
    dfgeoCountry = setCountryData(dfgeo,cols)
    return dfgeoCountry

# Manipulating school data
def getSchoolGeo(df,varname):
    df.columns = df.columns.str.split('(').str[0]
    df.columns = [varname + str(col) for col in df]
    statecols = varname + 'STATE NAME'
    discols   = varname + 'DISTRICT NAME'
    df      = df.rename(index=str, columns={statecols:'STATE NAME',discols:'DISTRICT NAME'})
    dfgeo   = getGeoData(df)
    return dfgeo

# School data manipulation

In [3]:
dataEl_2010_11['DISTRICT NAME'] = dataEl_2010_11['DISTRICT NAME'].str.title()
dataEl_2010_11.replace('ORISSA','ODISHA',inplace=True)
dataEl_2010_11.replace('JAMMU AND KASHMIR','JAMMU & KASHMIR',inplace=True)
dataEl_2010_11.replace('DELHI','NCT OF DELHI',inplace=True)
dataEl_2010_11.replace('ANDAMAN AND NICOBAR ISLANDS','ANDAMAN & NICOBAR ISLANDS',inplace=True)
dataEl_2010_11.replace('DAMAN AND DIU','DAMAN & DIU',inplace=True)

dataEl_2011_12['STATE NAME'] = dataEl_2011_12['STATE NAME'].str.upper()
dataEl_2011_12['DISTRICT NAME'] = dataEl_2011_12['DISTRICT NAME'].str.title()
dataEl_2011_12.replace('JAMMU AND KASHMIR','JAMMU & KASHMIR',inplace=True)
dataEl_2011_12.replace('DELHI','NCT OF DELHI',inplace=True)
dataEl_2011_12.replace('ORISSA','ODISHA',inplace=True)
dataEl_2011_12.replace('DAMAN AND DIU','DAMAN & DIU',inplace=True)
dataEl_2011_12.replace('D AND N HAVELI','DADRA & NAGAR HAVELI',inplace=True)
dataEl_2011_12.replace('A AND N ISLANDS','ANDAMAN & NICOBAR ISLANDS',inplace=True)


# schools 2010 in India

In [4]:
schoolgov2010 = dataEl_2010_11[['STATE NAME','DISTRICT NAME','PRIMARY (SCHGOVT1)','PRIMARY WITH UPPER PRIMARY (SCHGOVT2)','PRIMARY WITH UPPER PRIMARY SEC/HIGHER SEC. (SCHGOVT3)','UPPER PRIMARY ONLY (SCHGOVT4)','UPPER PRIMARY WITH SEC./HIGHER SEC. (SCHGOVT5)']]
schoolpvt2010 = dataEl_2010_11[['STATE NAME','DISTRICT NAME','PRIMARY (SCHPVT1)','PRIMARY WITH UPPER PRIMARY (SCHPVT2)','PRIMARY WITH UPPER PRIMARY SEC/HIGHER SEC. (SCHPVT3)','UPPER PRIMARY ONLY (SCHPVT4)','UPPER PRIMARY WITH SEC./HIGHER SEC. (SCHPVT5)']]

schoolgov2011 = dataEl_2011_12[['STATE NAME','DISTRICT NAME','PRIMARY (SCHGOVT1)','PRIMARY WITH UPPER PRIMARY (SCHGOVT2)','PRIMARY WITH UPPER PRIMARY SEC/HIGHER SEC. (SCHGOVT3)','UPPER PRIMARY ONLY (SCHGOVT4)','UPPER PRIMARY WITH SEC./HIGHER SEC. (SCHGOVT5)']]
schoolpvt2011 = dataEl_2011_12[['STATE NAME','DISTRICT NAME','PRIMARY (SCHPVT1)','PRIMARY WITH UPPER PRIMARY (SCHPVT2)','PRIMARY WITH UPPER PRIMARY SEC/HIGHER SEC. (SCHPVT3)','UPPER PRIMARY ONLY (SCHPVT4)','UPPER PRIMARY WITH SEC./HIGHER SEC. (SCHPVT5)']]

Schoolsgov2010geo = getSchoolGeo(schoolgov2010,'School_Government_2010_')
Schoolspvt2010geo = getSchoolGeo(schoolpvt2010,'School_Private_2010_')

Schoolsgov2011geo = getSchoolGeo(schoolgov2011,'School_Government_2011_')
Schoolspvt2011geo = getSchoolGeo(schoolpvt2011,'School_Private_2011_')

SchoolComb2010     = [Schoolsgov2010geo,Schoolspvt2010geo]
SchoolComgeo       =  reduce(lambda left,right: pd.merge(left,right, on=['geo_level','geo_code','name']),SchoolComb2010)
SchoolComgeo

SchoolComwazi = pd.melt(SchoolComgeo, id_vars=['geo_level', 'geo_code','name'], var_name='school', value_name="total")
SchoolComwazi = SchoolComwazi.sort_values(by=['geo_level','geo_code'])
SchoolComwazi['geo_version']  = '2011'
SchoolComwazi['year']  = SchoolComwazi['school'].str.split('_').str[2]
SchoolComwazi['type']  = SchoolComwazi['school'].str.split('_').str[1]
SchoolComwazi['schools'] = SchoolComwazi['school'].str.split('_').str[3]
SchoolComwazi['total']  = SchoolComwazi['total'].astype(int)
SchoolComwaziFinal = SchoolComwazi[['geo_level','geo_code','geo_version','schools','type','year','total']]
SchoolComwaziFinal = SchoolComwaziFinal.reset_index().drop(['index'],axis=1)
SchoolComwaziFinal.head()
#SchoolComwaziFinal.to_csv('data/school_type_2010.csv', index=False)

,geo_level,geo_code,geo_version,schools,type,year,total
0,country,IN,2011,PRIMARY WITH UPPER PRIMARY,Government,2010,186879
1,country,IN,2011,PRIMARY WITH UPPER PRIMARY SEC/HIGHER SEC.,Government,2010,10701
2,country,IN,2011,UPPER PRIMARY ONLY,Government,2010,142308
3,country,IN,2011,UPPER PRIMARY WITH SEC./HIGHER SEC.,Government,2010,48227
4,country,IN,2011,PRIMARY WITH UPPER PRIMARY,Private,2010,89110


# Schools 2011

In [12]:
SchoolComb2011     = [Schoolsgov2011geo,Schoolspvt2011geo]
SchoolComgeo       =  reduce(lambda left,right: pd.merge(left,right, on=['geo_level','geo_code','name']),SchoolComb2011)
SchoolComgeo

SchoolComwazi = pd.melt(SchoolComgeo, id_vars=['geo_level', 'geo_code','name'], var_name='school', value_name="total")
SchoolComwazi = SchoolComwazi.sort_values(by=['geo_level','geo_code'])
SchoolComwazi['geo_version']  = '2011'
SchoolComwazi['year']  = SchoolComwazi['school'].str.split('_').str[2]
SchoolComwazi['type']  = SchoolComwazi['school'].str.split('_').str[1]
SchoolComwazi['schools'] = SchoolComwazi['school'].str.split('_').str[3]
SchoolComwazi['total']  = SchoolComwazi['total'].astype(int)
SchoolComwaziFinal = SchoolComwazi[['geo_level','geo_code','geo_version','schools','type','year','total']]
SchoolComwaziFinal = SchoolComwaziFinal.reset_index().drop(['index'],axis=1)
SchoolComwaziFinal.head()
#SchoolComwaziFinal.to_csv('data/school_type_2011.csv', index=False)

,geo_level,geo_code,geo_version,schools,type,year,total
0,country,IN,2011,PRIMARY WITH UPPER PRIMARY,Government,2011,181526
1,country,IN,2011,PRIMARY WITH UPPER PRIMARY SEC/HIGHER SEC.,Government,2011,13099
2,country,IN,2011,UPPER PRIMARY ONLY,Government,2011,102860
3,country,IN,2011,UPPER PRIMARY WITH SEC./HIGHER SEC.,Government,2011,55739
4,country,IN,2011,PRIMARY WITH UPPER PRIMARY,Private,2011,81725
